### Importing Libraries

In [2]:
%matplotlib inline
import numpy as np
from numpy import empty
import warnings
warnings.filterwarnings('ignore')
import xarray as xr
import metpy as metpy
import netCDF4
from netCDF4 import Dataset
#from ctsm_py import utils  #when using utils in the CTSM_py directory (work); also need to do pip install -e
#import esmlab  #note: need to make sure to install in current environment first
#note that CMIP6 doesn't have metpy; use conda env:miniconda-analysis

### Defining simulation information

In [3]:
datadir        = "/glade/p/cgd/tss/people/dll/TRENDY2019_History/"
sim            = "S0_control/"
datadir        = datadir + sim
simname        = "TRENDY2019_S0_control_v2.clm2.h1."
var            = "GPP"
years          = "170001-201812"

In [4]:
print(datadir+simname+var+"."+years+".nc")

/glade/p/cgd/tss/people/dll/TRENDY2019_History/S0_control/TRENDY2019_S0_control_v2.clm2.h1.GPP.170001-201812.nc


In [5]:
data1          = xr.open_dataset(datadir+simname+var+"."+years+".nc")
#Use below for testing
#data1          = xr.open_dataset("/glade/scratch/dll/archive/TRENDY2019_S0_constant_v2/lnd/hist/TRENDY2019_S0_constant_v2.clm2.h1.2018-12.nc")

### Reading in variables from file

In [6]:
years          = np.arange(1700,2018,1)
month          = np.arange(1,12,1)
area           = data1.area
landfrac       = data1.landfrac
lat            = data1.lat
lon            = data1.lon
time           = data1.time

ixy            = data1.pfts1d_ixy
jxy            = data1.pfts1d_jxy
coltype        = data1.pfts1d_itype_col
vegtype        = data1.pfts1d_itype_veg
cellwtd        = data1.pfts1d_wtgcell

gpp            = data1.GPP
print(type(gpp))
print(gpp.dims)
#print (gpp['time'].values)


<class 'xarray.core.dataarray.DataArray'>
('time', 'pft')


### Processing variables

In [7]:
area.metpy.convert_units('m^2')
landarea       = area*landfrac
landarea.attrs['units']   = 'm^2'

num_years       = len(time)//12
print(num_years)
npftvector      = len(ixy)
type(cellwtd)

319


xarray.core.dataarray.DataArray

In [10]:
#print (cellwtd.values)
cellwt = cellwtd.values
cellwt_float =cellwt.astype(float)

#print (type(cellwt_float[1]))

nlat = len(lat.values)
nlon = len(lon.values)
#print(nlat)
#print(nlon)
nvegtype = len(vegtype.values)
ntim   = len(time.values)
print(ntim)
#print (nvegtype)
npft = (np.max(vegtype))
npft = npft.astype(int)
ntim2  = ntim//2

#print (npftvector)

3828


In [11]:
print (npft.values)
print(type(npft))
print(type(ntim))

pftlist = np.arange(0,(npft.values+1),1)
#print(pftlist)

77
<class 'xarray.core.dataarray.DataArray'>
<class 'int'>


In [12]:
print (vegtype.values[1])
gridded = empty([ntim,npft.values+1,nlat,nlon])

0.0


In [13]:
%%time
#this is time intensive: ~6.5s to regrid 1 month; 2.5min for timeseries
for i in range(npftvector):
    #gridded = empty([ntim,77,nlat,nlon])
    #print ('i = ',i)
    if (cellwt_float[i]>0):
        col     = ixy.values[i].astype(int) - 1
        row     = jxy.values[i].astype(int) - 1 
        vegidx = vegtype.values[i].astype(int)
        #print (gridded[:,vegidx, row, col])
        #print (gpp.values[:,i])
        gridded[:,vegidx, row, col] = gpp.values[:,i]
        #print (i)

CPU times: user 50.6 s, sys: 44.6 s, total: 1min 35s
Wall time: 2min 15s


### Adding dimensions and coordinates

In [15]:
print(gridded.shape)
grid_dims = xr.DataArray(gridded, dims=("time","pft","lat","lon"))
grid_dims = grid_dims.assign_coords(time=data1.time,pft=pftlist,lat=lat.values,lon=lon.values)
grid_dims.name = var
print(grid_dims)

(3828, 78, 192, 288)
<xarray.DataArray 'GPP' (time: 3828, pft: 78, lat: 192, lon: 288)>
array([[[[0., ..., 0.],
         ...,
         [0., ..., 0.]],

        ...,

        [[0., ..., 0.],
         ...,
         [0., ..., 0.]]],


       ...,


       [[[0., ..., 0.],
         ...,
         [0., ..., 0.]],

        ...,

        [[0., ..., 0.],
         ...,
         [0., ..., 0.]]]])
Coordinates:
  * lat      (lat) float32 -90.0 -89.057594 -88.11518 ... 89.057594 90.0
  * lon      (lon) float32 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.25 357.5 358.75
  * pft      (pft) int64 0 1 2 3 4 5 6 7 8 9 ... 68 69 70 71 72 73 74 75 76 77
  * time     (time) object 1700-02-01 00:00:00 ... 2019-01-01 00:00:00


### writing netCDF file

In [16]:
grid_dims.to_netcdf(path="/glade/scratch/dll/griddedExample.nc")

In [ ]:
#doesn't work yet... need to investigate
reshaped = grid_dims.values.reshape(num_years, 12, *gridded.values.shape[1:])
 
values = xr.DataArray(reshaped,
                      dims=('year','month','lat','lon'),
                      coords=(np.arange(1850, 1850+num_years), np.arange(12), data.lat, data.lon))
values.attrs['units'] = data.attrs['units']
sims[sim_name] = values